# Accessing International Debt Statistics (IDS) through World Bank Data API
## Part 1 - Query the API: indicators and locations
### *Python 3*

Date: 05/22/2019 <br>
[Data source](https://datacatalog.worldbank.org/dataset/international-debt-statistics)<br>
[Developer Information](https://datahelpdesk.worldbank.org/knowledgebase/topics/125589-developer-information)

This is first part of a two-part series on accessing the International Debt Statistics (IDS) database through the World Bank Data API. In this part, we will query the World Bank Data API to retrieve indicator names and location codes. In [Part 2](https://worldbank.github.io/debt-data/api-guide/ids-api-guide-python-2.html), we will use that information to explore the **regional trends of long-term external debt stocks** from the IDS database.

The following code in this guide will show step-by-step how to:

1. Setup up your environment with the needed Python packages
2. Create Indicator API Queries
3. Create Location API Queries

## 1. Setup
To start, make sure you have the following packages installed on your machine. If you aren't familiar with how to install a Python package, visit each of the linked packages for instructions.

- [requests](https://realpython.com/python-requests/): Making HTTP requests in Python
- [json](https://docs.python.org/3/library/json.html): Interpreting json

Then, open up your preferred mode of writing Python. This could be in a [Jupyter Notebook](https://jupyter.org/) using [Jupyter Lab](https://blog.jupyter.org/jupyterlab-is-ready-for-users-5a6f039b8906), using a code editor (like Atom or Visual Studio) + command line, or just from the command line. Then import the required packages:

In [1]:
# Import packages
import requests
import json

## 2. Indicator API Queries
To get a data series from the World Bank's API, you first need to use an "indicator code." For example, the indicator code for long-term external debt stock is "DT.DOD.DLXF.CD." These indicator codes can be found in the [World Bank Data Catalog](https://datacatalog.worldbank.org/), but you can also use the API to explore and select indicators. In this section, we will guide you through going from a topic idea to using the World Bank API to select an indictor code.

More information on this can also be found through the [Data Help Desk](https://datahelpdesk.worldbank.org/knowledgebase/articles/898599-indicator-api-queries).

First, we should choose a topic. Topics are high level categories to which all indicators are mapped, such as Education, Trade, etc. The topic in this guide is "External Debt," but we will need the topic's ID in order to see all its corresonponding indicator names and codes.

#### Getting the Topic ID for Debt

The following GET request will get us every topic from the World Bank Data API. However, the request is returned in a json format that is difficult to read as is, so we will include some code to parse through it. Then we can see each topic name and ID available in the World Bank Data API.

In [2]:
# Get all topic names from the World Bank API
topics = requests.get("http://api.worldbank.org/v2/topic?format=json")
topicsJSON = topics.json()
#print(responseJSON) # if you want to view the JSON response as is, remove the "#" at the beginning of the print command

# Parse through the response to see the topic names and ID numbers.
for i in topicsJSON[1]:
    print(i["id"],i["value"])

1 Agriculture & Rural Development
2 Aid Effectiveness
3 Economy & Growth
4 Education
5 Energy & Mining
6 Environment
7 Financial Sector
8 Health
9 Infrastructure
10 Social Protection & Labor
11 Poverty
12 Private Sector
13 Public Sector
14 Science & Technology
15 Social Development
16 Urban Development
17 Gender
18 Millenium development goals
19 Climate Change
20 External Debt
21 Trade


We can now see that "External Debt" topic has the ID "20"

#### Getting the Indicator code

Now that we have the ID for the topic "External Debt," we can make another request to the World Bank API to receive all of the indicator names and codes associated with that topic. In the code below, notice that the ID for "External Debt" is placed in the URL between "topic" and "indicator."

Most likely, there will be more External Debt indicators available than can fit on one page, the default is 50 results. So we will first print the "total" number of indicators. Then using that number we can adjust our API query.

In [3]:
# Requesting the indicators for the topic External Debt
indicators = requests.get("http://api.worldbank.org/v2/topic/20/indicator?format=json")
indicatorsJSON = indicators.json()

# Print the total number of indicators
print(indicatorsJSON[0]["total"])

229


The above code shows that there are 229 (UPDATE) indicators in the External Debt topic. When submitting our new query to the API, we will take this into account by setting the "per_page" parameter to 250. This will allow us to view all the results with one query, instead of having to request multiple pages. Then we can parse through the result to explore the different indicator names and corresponding IDs. Because there are so many indicators, the results are also saved as "ids-indicators.csv." (LINK NEEDED)

In [4]:
# Get all External Debt indicators, with a per_page parameter of 250.
indicators = requests.get("http://api.worldbank.org/v2/topic/20/indicator?per_page=250&format=json")
indicatorsJSON = indicators.json()
#print(indicatorsJSON) # to view ALL of the indicators as is, remove the "#" at the beginning of the print command

# Parse through the response to see the Indicator IDs and Names
for i in indicatorsJSON[1]:
    IDSindicators = (i["id"],i["name"])
    #print(IDSindicators) # to view the indicator ids and names, remove the "#" at the beginning of the print command

The resulting list gives us all of the External Debt indicators and their codes. The indicator and code we want is "DT.DOD.DLXF.CD External debt stocks, long-term (DOD, current US$)." Before using this data, we need to understand its full definition. You can use an API query to get that information as well.

In [5]:
# Use the indicator code to define x
x = "DT.DOD.DLXF.CD"

# Parse through the response to get the "sourceNote" or definition for the desired indicator
for dict_entity in indicatorsJSON[1]
    if dict_entity["id"] == x:
        print(dict_entity["sourceNote"])
    else:
        print("Try a different code")

Long-term debt is debt that has an original or extended maturity of more than one year. It has three components: public, publicly guaranteed, and private nonguaranteed debt. Data are in current U.S. dollars.


By using the API queries above, we were able sort through the World Bank API topics, find "External Debt," explore the corresponding indicators, and then select one indicator of interest and find its ID and definition.

## 2. Location API Queries

Now that we have the indicator code, we need to select a location. To select a location by country, region, or income level category you will need to know its 2 or 3 letter code. This section will show you how to use similar API queries to pull the location names and codes. 

For more information on this visit the [Data Help Desk](https://datahelpdesk.worldbank.org/knowledgebase/articles/898590-country-api-queries).

When pulling the list of locations, I will set the per_page result to 300, as there will too many results to display on one page.

In [6]:
# Requesting the locations
locations = requests.get("http://api.worldbank.org/v2/country?per_page=300&format=json")
locationsJSON = locations.json()
#print(locationsJSON) # if you want to view ALL of the indicators as is, remove the "#" at the beginning of the print command

From this list of locations, we need to pick one or more locations and then find the corresponding IDs and names. Because there are so many locations, the results are also saved as [location-codes csv](https://github.com/worldbank/debt-data/blob/master/location-codes.csv)*

###### *The location-codes csv was created using the API query: http://api.worldbank.org/v2/sources/2/country/data

In [7]:
# Parse through the response to see the location IDs and names
for i in locationsJSON[1]:
    locationCodes = (i["id"],i["name"])
    #print(locationCodes) # to view the location ids and names, remove the "#" at the beginning of the print command

From the resulting list, you can select the location IDs for the countries or regions of interest.

#### Now head to Part 2 of this series to find out how to use the indicator and location codes to call a data series from the World Bank Data API
### [Part 2 - Get and explore data](https://worldbank.github.io/debt-data/api-guide/ids-api-guide-python-2.html)